In [1]:
import sys
sys.path.append('/home/potzschf/repos/')
from helperToolz.helpsters import *
from helperToolz.evapo import *
import geopandas as gpd
import tarfile
from pyproj import CRS
from affine import Affine
from rasterio.enums import Resampling

workhorse = True

if workhorse:
    origin = 'Aldhani/eoagritwin/'
else:
    origin = ''

In [ ]:
# load evapo composites
evap_path = '/data/Aldhani/eoagritwin/et/Landsat/composites/Brandenburg/'
evaps = getFilelist(evap_path, '.tif', deep=True)

# get fields
fields = '/data/Aldhani/eoagritwin/fields/Auxiliary/grid_search/Brandenburg/quick_n_dirty/Fields_polygons.shp'
shp_ds = ogr.Open(fields)
layer = shp_ds.GetLayer()

/home/potzschf/mambaforge/envs/workhorse/lib/python3.12/site-packages/osgeo/ogr.py:601: FutureWarning: Neither ogr.UseExceptions() nor ogr.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(


In [3]:
# rasterize fields
dummy = gdal.Open(evaps[0])
dummy_x_size = dummy.RasterXSize
dummy_y_size = dummy.RasterYSize
dummy_gt = dummy.GetGeoTransform()
dummy_proj = dummy.GetProjection()

sub = gdal.GetDriverByName('MEM').Create('', dummy_x_size, dummy_y_size, 1, gdal.GDT_UInt32)
sub.SetGeoTransform(dummy_gt)
sub.SetProjection(dummy_proj)
band = sub.GetRasterBand(1)
band.SetNoDataValue(0)
gdal.RasterizeLayer(sub, [1], layer, options=["ATTRIBUTE=FieldID"])
sub_arr = sub.ReadAsArray()

In [4]:
# load in composites and stack em
ds_list = [gdal.Open(evap) for evap in evaps]
raster_list = [ds.GetRasterBand(1).ReadAsArray() for ds in ds_list]
block = np.dstack(raster_list)

In [5]:
years = [evap.split('Brandenburg')[1].split('/')[1] for evap in evaps]
months = [evap.split('_')[-2] for evap in evaps]

In [14]:
# zone_ids = np.unique(sub_arr)
# zone_ids = zone_ids[zone_ids != 0]  # remove background if 0
zone_ids = [114280, 117368, 120739]
results = []
print(f'total zones: {zone_ids}')
for zone_id in zone_ids:
    mask = sub_arr == zone_id

    for band_idx in range(1, block.shape[2]):
        band_stats = {
            "zone_id": zone_id,
            "Year": years[band_idx],
            "month": months[band_idx]
        }

        band = block[:, :, band_idx]
        values = band[mask]
        values = values[~np.isnan(values)]  # optional: remove NaNs

        if values.size > 0:
            band_stats["band_mean"] = values.mean()
            band_stats["band_std"] = values.std()
            band_stats["band_min"] = values.min()
            band_stats["band_max"] = values.max()
        else:
            band_stats["band_mean"] = np.nan
            band_stats["band_std"] = np.nan
            band_stats["band_min"] = np.nan
            band_stats["band_max"] = np.nan
        
        results.append(band_stats)


total zones: [114280, 117368, 120739]


In [15]:
df = pd.DataFrame(results)
df.to_csv('/data/Aldhani/eoagritwin/et/Landsat/composites/Brandenburg/field_stats_3ids.csv', index=False)

In [16]:
df

,zone_id,Year,month,band_mean,band_std,band_min,band_max
0,114280,2018,02,0.422121,0.056133,0.308235,0.553493
1,114280,2018,03,0.469640,0.077726,0.307107,0.693715
2,114280,2018,04,1.460047,0.180498,0.963402,2.023805
3,114280,2018,05,3.331038,0.134298,2.709492,3.771546
4,114280,2018,06,3.131817,0.727405,1.823650,4.429602
...,...,...,...,...,...,...,...
244,120739,2024,08,4.147490,0.289099,3.112500,4.564156
245,120739,2024,09,0.777176,0.366046,0.169345,1.881888
246,120739,2024,10,0.724376,0.131977,0.260323,1.052535
247,120739,2024,11,NaN,NaN,NaN,NaN
